In [5]:
# importar
import re, json
import pandas as pd
import os
import numpy as np
import requests

In [6]:
# adicionar csv
df = pd.read_csv('searchbar.csv')
#renomear colunas
df.columns = ['player_position', 'url']
df

,player_position,url
0,R. Owusu-Oduro (Goalkeeper),https://datamb.football/R_Owusu_Oduro_Goalkeep...
1,V. Jaroš (Goalkeeper),https://datamb.football/V_Jaro__Goalkeeper_stats
2,André Gomes (Goalkeeper),https://datamb.football/Andre_Gomes_Goalkeeper...
3,H. Koffi (Goalkeeper),https://datamb.football/H_Koffi_Goalkeeper_stats
4,João Valido (Goalkeeper),https://datamb.football/Jo_o_Valido_Goalkeeper...
...,...,...
2252,Volendam (Team),https://datamb.football/Volendam_Team_stats
2253,Werder Bremen (Team),https://datamb.football/Werder_Bremen_Team_stats
2254,West Ham United (Team),https://datamb.football/West_Ham_United_Team_s...
2255,Wolfsburg (Team),https://datamb.football/Wolfsburg_Team_stats


In [7]:
# conferir os times
df.iloc[2125:2135]

,player_position,url
2125,Ajax (Team),https://datamb.football/Ajax_Team_stats
2126,Alverca (Team),https://datamb.football/Alverca_Team_stats
2127,Angers SCO (Team),https://datamb.football/Angers_SCO_Team_stats
2128,Arouca (Team),https://datamb.football/Arouca_Team_stats
2129,Arsenal (Team),https://datamb.football/Arsenal_Team_stats
2130,Aston Villa (Team),https://datamb.football/Aston_Villa_Team_stats
2131,Atalanta (Team),https://datamb.football/Atalanta_Team_stats
2132,Athletic Club (Team),https://datamb.football/Athletic_Club_Team_stats
2133,Atlético Madrid (Team),https://datamb.football/Atletico_Madrid_Team_s...
2134,Augsburg (Team),https://datamb.football/Augsburg_Team_stats


In [8]:
# players | teams
df_players = df.iloc[:2127].copy()
df_teams = df.iloc[2127:].copy()
df_players.reset_index(drop=True, inplace=True)
df_teams.reset_index(drop=True, inplace=True)


In [9]:
# conferir tabelas
print(df_players.shape)
print(df_teams.shape)

df_players.head(2)

(2127, 2)
(130, 2)


,player_position,url
0,R. Owusu-Oduro (Goalkeeper),https://datamb.football/R_Owusu_Oduro_Goalkeep...
1,V. Jaroš (Goalkeeper),https://datamb.football/V_Jaro__Goalkeeper_stats


In [10]:
# padronizar colunas tabela teams
df_teams.columns = ['teams', 'url']
df_teams.head(2)

,teams,url
0,Angers SCO (Team),https://datamb.football/Angers_SCO_Team_stats
1,Arouca (Team),https://datamb.football/Arouca_Team_stats


In [11]:
# salvar tabelas separadas
df_players.to_csv("players_only.csv", index=False)
df_teams.to_csv("teams_only.csv", index=False)


In [12]:
# times não tem posição, jogadores sim
mask_players = df["player_position"].str.contains(r"\(", na=False)
df_players = df[mask_players].copy()
df_teams = df[~mask_players].copy()


In [13]:
# test scrapping
test_url = df_players.loc[0, 'url']
test_url

'https://datamb.football/R_Owusu_Oduro_Goalkeeper_stats'

In [14]:
# teste mais simples, retorna 200 = ok
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9"
}

url = df_players.loc[0, "url"]

response = requests.get(url, headers=headers)
response.status_code


200

In [15]:
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    )
}

url = df_players.loc[0, "url"]
html = requests.get(url, headers=headers).text


In [16]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")
scripts = soup.find_all("script")

len(scripts)

14

In [20]:
stats_blocks = soup.find_all("div", class_="stat")
len(stats_blocks)


0

In [21]:
metrics = {}

for block in stats_blocks:
    label = block.find("span", class_="label").text.strip()
    value = block.find("span", class_="value").text.strip()
    metrics[label] = value

metrics


{}

In [22]:
import requests

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    )
}

url = df_players.loc[0, "url"]
html = requests.get(url, headers=headers).text


In [23]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")
scripts = soup.find_all("script")

len(scripts)


14

In [24]:
for i, s in enumerate(scripts):
    if len(s.text) > 5000:
        print(i, len(s.text))


11 8051
12 21225


In [25]:
import re, json

script_text = scripts[idx].text

# pega o primeiro grande objeto {...}
match = re.search(r"\{.*\}", script_text, re.S)

data = json.loads(match.group())


NameError: name 'idx' is not defined